In [106]:
# all necessary libraries
import pandas as pd
import sys
from pathlib import Path

current_file_dir = Path(r"scripts\machine_learning").resolve().parent
parent_dir = current_file_dir.parent.parent
sys.path.insert(0, str(parent_dir))

In [107]:
df_dist = pd.read_csv('../../data/features/afp_distance_features.csv')
df_tree = pd.read_csv('../../data/features/medoid_tree_features.csv')

# 3. Merge on sequence_id
df = df_dist.merge(df_tree, on=['sequence_id', 'label'])

motif_paths = [
    r"..\..\data\motifs\afgp_motifs.txt",
    r"..\..\data\motifs\type1_motifs.txt",
    r"..\..\data\motifs\type2_motifs.txt",
    r"..\..\data\motifs\type3_motifs.txt",
    r"..\..\data\motifs\type4_motifs.txt"
]

positive_reviewed_sequences_path = r"..\..\data\positives\afp_all_c90.faa"
positive_non_reviewed_sequences_path = r""
negative_sequences_path = r"..\..\data\negatives\non_afp_c90.faa"

# mappings for HMM
amino_acids = 'ARNDCQEGHILKMFPSTWYV' # Standard 20 amino acids
aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
int_to_aa = {i: aa for aa, i in aa_to_int.items()}

if 'N' in aa_to_int: aa_to_int['B'] = aa_to_int['N'] # B -> N
if 'A' in aa_to_int: aa_to_int['X'] = aa_to_int['A'] # X -> A
if 'C' in aa_to_int: aa_to_int['U'] = aa_to_int['C'] # U -> C
if 'E' in aa_to_int: aa_to_int['Z'] = aa_to_int['E'] # Z -> E

In [108]:
df

,sequence_id,label,length,net_charge_per_residue,hydrophobic_percent,polar_percent,charged_percent,gly_percent,pro_percent,ambiguous_count,...,medoid_type4_gap_rate,medoid_afgp_score,medoid_afgp_identity,medoid_afgp_coverage,medoid_afgp_gap_rate,medoid_top1_type,medoid_top1_identity,medoid_delta_top1_top2_ident,medoid_top1_type_by_score,medoid_delta_top1_top2_score
0,P04002,AFP,82,-0.012195,70.731707,15.853659,10.975610,2.439024,6.097561,0,...,0.0,-290.5,0.768293,1.000000,0.0,type1,0.888889,0.120596,type1,153.0
1,P24856,AFP,790,0.001266,72.784810,26.202532,0.126582,0.126582,9.240506,0,...,0.0,1580.0,1.000000,1.000000,0.0,afgp,1.000000,0.197802,afgp,1848.0
2,B1P0S1,AFP,218,-0.004587,72.935780,17.889908,8.715596,0.458716,0.917431,0,...,0.0,-112.5,0.774194,0.995413,0.0,type1,0.857143,0.082949,type1,126.0
3,A0ZT93,AFP,168,-0.029762,48.214286,17.857143,17.261905,7.142857,4.166667,0,...,0.0,-329.0,0.425000,0.952381,0.0,type2,0.703704,0.144382,type2,211.5
4,P05140,AFP,163,-0.036810,47.239264,23.312883,13.496933,6.748466,5.521472,0,...,0.0,-317.5,0.455128,0.957055,0.0,type2,1.000000,0.392857,type2,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4733,Q7SZF1,NON_AFP,439,-0.072893,43.735763,21.640091,25.056948,3.189066,6.833713,0,...,0.0,-350.5,0.320312,0.874715,0.0,type3,0.734375,0.159662,type2,0.0
4734,Q7T012,NON_AFP,242,0.008264,42.975207,21.487603,22.314050,7.438017,4.545455,0,...,0.0,-346.5,0.371681,0.933884,0.0,type3,0.610169,0.087234,type4,9.0
4735,Q7T2B3,NON_AFP,499,0.028056,34.268537,31.062124,25.651303,5.611222,6.212425,0,...,0.0,-374.5,0.284404,0.873747,0.0,type3,0.765625,0.090828,type4,4.5
4736,Q7ZVJ8,NON_AFP,206,-0.004854,47.087379,25.728155,16.990291,6.310680,5.825243,0,...,0.0,-333.0,0.450262,0.927184,0.0,type3,0.555556,0.017094,type4,16.5


## DATA PREPARATION

#### need to parse the txt motifs so that it is in list format (done)

In [109]:
import os

motif_dict = {}

for path in motif_paths:
    key = os.path.splitext(os.path.basename(path))[0]
    motif_dict[key] = []

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or set(line) == {"="}:
                continue
            motif_dict[key].append(line)


In [110]:
motif_dict

{'afgp_motifs': ['VTAAPAATAATAATPATAALNFAATAATPATPATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAAAAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAACNFAATAAT',
  'ATAAT',
  'IFAA',
  'AATAAT',
  'ATAA',
  'FAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATPAFNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALHFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAATPALIFAATAATAATPATPATPALIFAATAATAATPATPALNFAATAATAATTAARG'],
 'type1_motifs': ['AVTA', 'NAASAAA', 'AAAVAAATLEAAAAKAAATAVSAAAAAAAAAIAFAAAP'],
 'type2_motifs': ['LLVCAM', 'PVFR', 'LASIHS', 'WIGGS', 'VIPEVTPPSIM'],
 'type3_motifs'

#### need to grab access to each individual sequence so we can check if the motifs are within the sequence

In [111]:
df["type1_motifs"] = None
df["type2_motifs"] = None
df["type3_motifs"] = None
df["type4_motifs"] = None
df["afgp_motifs"] = None

In [112]:
df["type1_motifs"]

0       None
1       None
2       None
3       None
4       None
        ... 
4733    None
4734    None
4735    None
4736    None
4737    None
Name: type1_motifs, Length: 4738, dtype: object

In [113]:
# dictionary that saves all the data
def parse_fasta_to_dict(path):
    sequences = {}
    current_key = None
    current_seq = []

    with open(path, "r") as f:
        for line in f:
            line = line.strip()

            if not line:
                continue

            # Header line
            if line.startswith(">"):
                # Save previous entry if exists
                if current_key is not None:
                    sequences[current_key] = "".join(current_seq)

                # Extract the code between the first two "|"
                parts = line.split("|")
                current_key = parts[1]  # e.g. "P86268"
                current_seq = []
            else:
                # Sequence line
                current_seq.append(line)

        # Save the last sequence
        if current_key is not None:
            sequences[current_key] = "".join(current_seq)

    return sequences


In [114]:
positive_sequences = parse_fasta_to_dict(positive_reviewed_sequences_path)
negative_sequences = parse_fasta_to_dict(negative_sequences_path)

In [115]:
positive_sequences

{'P04002': 'MALSLFTVGQLIFLFWTMRITEASPDPAAKAAPAAAAAPAAAAPDTASDAAAAAALTAANAKAAAELTAANAAAAAAATARG',
 'P24856': 'VTAAPAATAATAATPATAALNFAATAATPATPATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAAAAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAACNFAATAATPATAATPALIFAATAATAATPATAACNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATPAFNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALHFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAATPALIFAATAATAATPATPATPALIFAATAATAATPATPALNFAATAATAATTAARG',
 'B1P0S1': 'MALSLFTVGQFIFLFWTISITEANIDPAARAAAAAAASKAAVTAADAAAAAATIAASAASVAAATAADDAAASIATINAASAAAKS

In [116]:
negative_sequences

{'A0A0C5PHQ7': 'MEILNQRLNQQFDSWMGPRDPRVRGWLLLDNYLPTLSFTIIYLLIVWMGPKYMRNRQPVSCRGILVVYNMALTLLSLYMFYELVTAVWQGGYNFFCQDTHSGGEADNRVINVLWWYYFSKLIEFMDTFFFILRKNNHQITFLHIYHHFTMLNIWWFVMNWVPCGHSYFGATFNSFIHVLMYSYYGLSAIPAIQPYLWWKKYITQGQLVQFVLTMIQTSCAVVWPCGFPKGWLYFQISYMITLIILFSNFYIQTYKKKGTAAKKDPRHNGIKSVNGHSNGASHTNAVKNRKARTD',
 'A0A0G2KQY6': 'MTLRRASGCRQLTLTIGLALTLGLLQWPIGDVRGQDGASPAQVLQELLTRYGDNASISVPQLRSLLVRLNGGQSEDHDSKTQPTRTNASKCLAADTLAVYGMSEQSRIDERGLQQICPTMIQQLDSQACKTQPNQESESSPRPTEAEVWGYGLLCVTVISLCSLVGASVVPFMRKTFYKRLLLYFIALAIGTLYSNALFQLIPEAFGFDPMEDYYVPKSAVVFGGFYLFFFTEKILKMILKPKDTGGHGHGHSHFPAERYANSNGDLEDGVMEKLQNGEAGGAALPRAEADGRGVGEDDKMLSTGQTVQDTQSSGGGGTGGCYWLKGRAYSDIGTLAWMITLSDGLHNFIDGLAIGASFTASVFQGISTSVAILCEEFPHELGDFVILLNAGMSIQQALFFNFLSACCCYLGMGFGILAGNNFSPNWIFALAGGMFLYIALADMFPEMNEVSREEEEAGGSGFLLTFALQNAGLLTGFAIMLVLTIYSGQIQLG',
 'A0A0G2KTI4': 'MSASPPISAGDYLSAPEPDALKPAGPTPSQSRFQVDLVTESAGDGETTVGFDSSPPEYVAEPPPDGLRDSVSGGEEAKGRFRVVNFAASSPDAAPAETAQNGDTVMSEGSLHSSTGGQQHHHYDTHTNTYYLRTFGHNTIDAVPKIDFYRQTAAPLGE

In [117]:
# Itterate through all the rows in the df, 
# check the sequence_id column of the df 
# find the corresponding match in either negative_sequences or positive_sequences
# then check to find if any motif in motif_dict matches
# if there is a match, we want to fill these columns with either ones or zeros
# df["type1_motifs"] = None
# df["type2_motifs"] = None
# df["type3_motifs"] = None
# df["type4_motifs"] = None
# df["afgp_motifs"] = None
# so one row might have a match in both typ3 and afgp but no matches for other AFPs 
# It would look something like this for these columns:
# type1_motif type2_motif type3_motif type4_motif afgp_motif
# 0    0    1    0      1

# Map motif keys from motif_dict to df column names
motif_columns = ["type1_motifs", "type2_motifs", "type3_motifs", "type4_motifs", "afgp_motifs"]

# Ensure columns are initialized to 0
for col in motif_columns:
    df[col] = 0

# Iterate through DataFrame rows
for idx, row in df.iterrows():
    seq_id = row["sequence_id"]

    # Determine if seq_id is in positives or negatives
    if seq_id in positive_sequences:
        seq = positive_sequences[seq_id]
    elif seq_id in negative_sequences:
        seq = negative_sequences[seq_id]
    else:
        # No matching sequence → skip
        continue

    # Check motifs for every AFP type
    for motif_key in motif_columns:
        motifs = motif_dict[motif_key]

        # Check if any motif appears in the sequence
        has_match = any(m in seq for m in motifs)
        df.at[idx, motif_key] = 1 if has_match else 0


In [118]:
df.columns

Index(['sequence_id', 'label', 'length', 'net_charge_per_residue',
       'hydrophobic_percent', 'polar_percent', 'charged_percent',
       'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate',
       'top1_type', 'top1_identity', 'top1_score', 'top1_coverage',
       'top2_type', 'top2_identity', 'delta_top1_top2', 'type1_score',
       'type1_aligned_len', 'type1_coverage', 'type1_identity',
       'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match',
       'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity',
       'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match',
       'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity',
       'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match',
       'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity',
       'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_match',
       'afgp_score', 'afgp_aligned_len', 'afgp_co

#### Hydrophobicity and steric hinderance

In [119]:


from sequence_analysis.extract_features import steric_hindrance_score, hydrophobicity_score

# Create new columns
df["hydrophobicity"] = None
df["steric_hindrance"] = None

for idx, row in df.iterrows():
    seq_id = row["sequence_id"]

    # find sequence
    if seq_id in positive_sequences:
        seq = positive_sequences[seq_id]
    elif seq_id in negative_sequences:
        seq = negative_sequences[seq_id]
    else:
        # sequence not found
        continue

    # compute scores
    hydro_score = hydrophobicity_score(seq)
    steric_score = steric_hindrance_score(seq)

    # assign to dataframe
    df.at[idx, "hydrophobicity"] = hydro_score
    df.at[idx, "steric_hindrance"] = steric_score


#### Integrating non reviewed positive data (PBLAST) (NOT DONE YET)

In [120]:
# Create the new column 'detailed_label' for stratification purposes
df['detailed_label'] = df['label'].apply(
    lambda x: 'reviewed' if x == 'AFP' else 'negative'
)

# Show a sample of the new column
print("--- DataFrame with New Column ---")
print(df[['label', 'detailed_label']].head())

# Show the distribution of the new column
print("\n--- Distribution of detailed_label ---")
print(df['detailed_label'].value_counts())


--- DataFrame with New Column ---
  label detailed_label
0   AFP       reviewed
1   AFP       reviewed
2   AFP       reviewed
3   AFP       reviewed
4   AFP       reviewed

--- Distribution of detailed_label ---
detailed_label
negative    4706
reviewed      32
Name: count, dtype: int64


In [121]:
# MAKE NUMBER OF POSITIVE AND NEGATIVE SEQUENCES THE SAME
import pandas as pd

# Count how many AFP rows we have
num_afp = df[df["label"] == "AFP"].shape[0]

# Filter AFP and NON_AFP subsets
df_afp = df[df["label"] == "AFP"]
df_non_afp = df[df["label"] == "NON_AFP"]

# Randomly sample NON_AFP to match AFP count
df_non_afp_sampled = df_non_afp.sample(n=num_afp, random_state=42)

# Combine them into a balanced dataset
df_balanced = pd.concat([df_afp, df_non_afp_sampled], axis=0).sample(frac=1, random_state=42)

# Reset index for cleanliness
df_balanced = df_balanced.reset_index(drop=True)


In [122]:
df = df_balanced

#### ONE HOT ENCODING SOME FEATURES

In [123]:
# 1. Identify non-numeric columns
non_numeric_cols = df.select_dtypes(exclude=['float', 'int']).columns.tolist()

non_numeric_cols = ["top1_type", "top2_type", "medoid_top1_type",	"medoid_top1_type_by_score"]

df[non_numeric_cols]


,top1_type,top2_type,medoid_top1_type,medoid_top1_type_by_score
0,type4,type3,type3,type4
1,type3,type4,type3,type4
2,type1,afgp,type1,type1
3,type3,type4,type3,type2
4,type3,type4,type3,type3
...,...,...,...,...
59,type3,type4,type3,type2
60,type1,afgp,type1,type1
61,type3,type2,type3,type4
62,type3,type4,type3,type2


In [124]:
# 3. One-hot encode them
df = pd.get_dummies(df, columns=non_numeric_cols)
df.drop(["label","medoid_type1_id"	,"medoid_type2_id",	"medoid_type3_id"	,"medoid_type4_id",	"medoid_afgp_id"], inplace=True, axis=1)

In [125]:
df

,sequence_id,length,net_charge_per_residue,hydrophobic_percent,polar_percent,charged_percent,gly_percent,pro_percent,ambiguous_count,ambiguous_rate,...,medoid_top1_type_afgp,medoid_top1_type_type1,medoid_top1_type_type2,medoid_top1_type_type3,medoid_top1_type_type4,medoid_top1_type_by_score_afgp,medoid_top1_type_by_score_type1,medoid_top1_type_by_score_type2,medoid_top1_type_by_score_type3,medoid_top1_type_by_score_type4
0,Q32PV0,300,0.000000,42.333333,19.333333,28.666667,5.666667,7.666667,0,0.0,...,False,False,False,True,False,False,False,False,False,True
1,P86880,159,-0.050314,45.911950,21.383648,21.383648,6.289308,3.144654,0,0.0,...,False,False,False,True,False,False,False,False,False,True
2,P04002,82,-0.012195,70.731707,15.853659,10.975610,2.439024,6.097561,0,0.0,...,False,True,False,False,False,False,True,False,False,False
3,F1Q930,377,-0.026525,46.949602,19.098143,25.464191,3.978780,6.631300,0,0.0,...,False,False,False,True,False,False,False,True,False,False
4,P07457,87,0.034483,55.172414,25.287356,10.344828,6.896552,6.896552,0,0.0,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Q803U7,806,0.013648,36.476427,27.047146,25.682382,6.947891,6.575682,0,0.0,...,False,False,False,True,False,False,False,True,False,False
60,P09031,97,-0.030928,72.164948,12.371134,13.402062,2.061856,4.123711,0,0.0,...,False,True,False,False,False,False,True,False,False,False
61,Q76CE8,243,0.020576,52.674897,27.572016,7.818930,11.934156,4.115226,0,0.0,...,False,False,False,True,False,False,False,False,False,True
62,B5X337,366,0.038251,54.371585,20.218579,14.207650,4.644809,5.464481,0,0.0,...,False,False,False,True,False,False,False,True,False,False


In [126]:
df["detailed_label"]

0     negative
1     negative
2     reviewed
3     negative
4     reviewed
        ...   
59    negative
60    reviewed
61    reviewed
62    negative
63    negative
Name: detailed_label, Length: 64, dtype: object

## TRAINING THE MODEL

#### CROSS VALIDATION INFO

In [127]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from hmmlearn.hmm import CategoricalHMM

# Helper function to convert amino acid sequences to numerical arrays
def sequence_to_array(sequence, aa_to_index):
    """Convert amino acid sequence string to numerical array"""
    return np.array([[aa_to_index.get(aa, 0)] for aa in sequence])

def prepare_hmm_data(sequence_ids, sequence_dict, aa_to_index):
    """
    Prepare sequences and lengths for HMM training from sequence IDs
    
    Parameters:
    -----------
    sequence_ids : array-like
        List of sequence IDs to include
    sequence_dict : dict
        Dictionary mapping sequence_id to sequence string
    aa_to_index : dict
        Dictionary mapping amino acid to index
        
    Returns:
    --------
    X : array, shape (n_samples, 1)
        Concatenated numerical sequences
    lengths : array
        Length of each sequence
    """
    sequences = []
    lengths = []
    
    for seq_id in sequence_ids:
        if seq_id in sequence_dict:
            seq_str = sequence_dict[seq_id]
            seq_array = sequence_to_array(seq_str, aa_to_index)
            sequences.append(seq_array)
            lengths.append(len(seq_array))
    
    if len(sequences) == 0:
        return np.array([]).reshape(0, 1), np.array([], dtype=np.int32)
    
    X = np.vstack(sequences)
    lengths = np.array(lengths, dtype=np.int32)
    
    return X, lengths

# Create stratification labels and binary target
stratify_labels = df['detailed_label'].values
df['Binary_Target'] = df['detailed_label'].apply(
    lambda x: 0 if x == 'negative' else 1
)
y = df['Binary_Target'].values

# Ensure sequence_id column exists in df
if 'sequence_id' not in df.columns:
    raise ValueError("DataFrame must contain 'sequence_id' column")

# Initialize StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# List to store evaluation scores
auc_scores = []
models = []

# Optimal number of components from your HMM hyperparameter tuning
n_components = 6

print(f"ready {n_splits}-fold Stratified Cross-Validation with HMM features...")


ready 5-fold Stratified Cross-Validation with HMM features...


#### TRAINING LOOP

In [128]:

# --- Cross-Validation Loop ---
for fold, (train_index, test_index) in enumerate(skf.split(df, stratify_labels)):
    print(f"\n{'='*70}")
    print(f"FOLD {fold+1}/{n_splits}")
    print(f"{'='*70}")
    
    # Split data for this fold
    df_train = df.iloc[train_index].copy()
    df_test = df.iloc[test_index].copy()
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Get sequence IDs for training set
    train_sequence_ids = df_train['sequence_id'].values
    
    # Separate positive and negative sequence IDs in training set
    train_positive_ids = df_train[df_train['Binary_Target'] == 1]['sequence_id'].values
    train_negative_ids = df_train[df_train['Binary_Target'] == 0]['sequence_id'].values
    
    print(f"Training set: {len(train_positive_ids)} positive, {len(train_negative_ids)} negative sequences")
    
    # --- Train HMM Models ---
    print("\nTraining HMM models...")
    
    # Prepare positive HMM training data
    positive_X, positive_lengths = prepare_hmm_data(
        train_positive_ids, 
        positive_sequences, 
        aa_to_int
    )
    
    # Prepare negative HMM training data
    negative_X, negative_lengths = prepare_hmm_data(
        train_negative_ids, 
        negative_sequences, 
        aa_to_int
    )
    
    print(f"Positive HMM data: {positive_X.shape[0]} total samples, {len(positive_lengths)} sequences")
    print(f"Negative HMM data: {negative_X.shape[0]} total samples, {len(negative_lengths)} sequences")
    
    # Train positive model
    positive_model = CategoricalHMM(
        n_components=n_components,
        n_features=20,
        n_iter=100,
        random_state=42
    )
    positive_model.fit(positive_X, positive_lengths)
    
    # Train negative model
    negative_model = CategoricalHMM(
        n_components=n_components,
        n_features=20,
        n_iter=100,
        random_state=42
    )
    negative_model.fit(negative_X, negative_lengths)
    
    print("HMM models trained successfully!")
    
    # --- Generate HMM Features for Training Set ---
    print("\nGenerating HMM features for training set...")
    
    train_positive_log_probs = []
    train_negative_log_probs = []
    
    for seq_id in df_train['sequence_id']:
        # Try to find sequence in positive or negative dictionary
        if seq_id in positive_sequences:
            seq_str = positive_sequences[seq_id]
        elif seq_id in negative_sequences:
            seq_str = negative_sequences[seq_id]
        else:
            train_positive_log_probs.append(np.nan)
            train_negative_log_probs.append(np.nan)
            continue
        
        seq_array = sequence_to_array(seq_str, aa_to_int)
        
        # Score with both models
        pos_log_prob = positive_model.score(seq_array)
        neg_log_prob = negative_model.score(seq_array)
        
        train_positive_log_probs.append(pos_log_prob)
        train_negative_log_probs.append(neg_log_prob)
    
    # Add HMM features to training dataframe
    df_train['hmm_positive_log_prob'] = train_positive_log_probs
    df_train['hmm_negative_log_prob'] = train_negative_log_probs
    
    # --- Generate HMM Features for Test Set ---
    print("Generating HMM features for test set...")
    
    test_positive_log_probs = []
    test_negative_log_probs = []
    
    for seq_id in df_test['sequence_id']:
        if seq_id in positive_sequences:
            seq_str = positive_sequences[seq_id]
        elif seq_id in negative_sequences:
            seq_str = negative_sequences[seq_id]
        else:
            test_positive_log_probs.append(np.nan)
            test_negative_log_probs.append(np.nan)
            continue
        
        seq_array = sequence_to_array(seq_str, aa_to_int)
        
        pos_log_prob = positive_model.score(seq_array)
        neg_log_prob = negative_model.score(seq_array)
        
        test_positive_log_probs.append(pos_log_prob)
        test_negative_log_probs.append(neg_log_prob)
    
    # Add HMM features to test dataframe
    df_test['hmm_positive_log_prob'] = test_positive_log_probs
    df_test['hmm_negative_log_prob'] = test_negative_log_probs
    
    # --- Prepare Features for Logistic Regression ---
    # Drop columns that shouldn't be features
    columns_to_drop = ['detailed_label', 'Binary_Target', 'sequence_id']
    
    X_train = df_train.drop(columns=columns_to_drop)
    X_test = df_test.drop(columns=columns_to_drop)
    
    # Handle any NaN values (if sequences were missing)
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())  # Use training mean for test set
    
    print(f"\nTraining features shape: {X_train.shape}")
    print(f"Test features shape: {X_test.shape}")
    print(f"Features: {list(X_train.columns)}")
    
    # --- Train Logistic Regression ---
    print("\nTraining Logistic Regression model...")
    
    model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    models.append(model)
    
    # Predict probabilities on the test set
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluate using AUC-ROC
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(roc_auc)
    
    print(f"\nFold {fold+1} Test AUC-ROC: {roc_auc:.4f}")
    
    # Show feature importances (coefficients)
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'coefficient': model.coef_[0]
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10).to_string(index=False))

# --- Final Results ---
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)
print(f"Individual Fold AUC-ROC Scores: {[f'{score:.4f}' for score in auc_scores]}")
print(f"Mean AUC-ROC: {np.mean(auc_scores):.4f}")
print(f"Std Dev of AUC-ROC: {np.std(auc_scores):.4f}")


FOLD 1/5
Training set: 26 positive, 25 negative sequences

Training HMM models...
Positive HMM data: 3831 total samples, 26 sequences
Negative HMM data: 9499 total samples, 25 sequences
HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (51, 103)
Test features shape: (13, 103)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_r

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (51, 103)
Test features shape: (13, 103)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (51, 103)
Test features shape: (13, 103)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (51, 103)
Test features shape: (13, 103)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (52, 103)
Test features shape: (12, 103)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\2504911493.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set
